# Homework Starter — Stage 04: Data Acquisition and Ingestion
Name: Jingdong Yang
Date: 8.17

## Objectives
- API ingestion with secrets in `.env`
- Scrape a permitted public table
- Validate and save raw data to `data/raw/`

In [22]:
import os, json, time, datetime as dt, csv, pathlib
from typing import Dict, List
import requests
import pandas as pd
from bs4 import BeautifulSoup
from dotenv import load_dotenv


RAW = pathlib.Path("data/raw")
RAW.mkdir(parents=True, exist_ok= True)

load_dotenv()
ALPHA_KEY = os.getenv("ALPHAVANTAGE_API_KEY")
print("Loaded APLHAVANTAGE_API_KEY?", bool(ALPHA_KEY))
print(ALPHA_KEY)

Loaded APLHAVANTAGE_API_KEY? True
9QRBQEYG63EDDPKL


## Helpers (use or modify)

In [2]:
def ts():
    return dt.datetime.now().strftime('%Y%m%d-%H%M%S')

def save_csv(df: pd.DataFrame, prefix: str, **meta):
    mid = '_'.join([f"{k}-{v}" for k,v in meta.items()])
    path = RAW / f"{prefix}_{mid}_{ts()}.csv"
    df.to_csv(path, index=False)
    print('Saved', path)
    return path

def validate(df: pd.DataFrame, required):
    missing = [c for c in required if c not in df.columns]
    return {'missing': missing, 'shape': df.shape, 'na_total': int(df.isna().sum().sum())}

## Part 1 — API Pull (Required)
Choose an endpoint (e.g., Alpha Vantage or use `yfinance` fallback).

In [12]:
SYMBOL = 'AAPL'
USE_ALPHA = bool(os.getenv('ALPHAVANTAGE_API_KEY'))
if USE_ALPHA:
    url = 'https://www.alphavantage.co/query'
    params = {'function':'TIME_SERIES_DAILY','symbol':SYMBOL,'outputsize':'compact','apikey':os.getenv('ALPHAVANTAGE_API_KEY')}
    r = requests.get(url, params=params, timeout=30)
    r.raise_for_status()
    js = r.json()
    key = [k for k in js if 'Time Series' in k][0]
    df_api = pd.DataFrame(js[key]).T.reset_index().rename(columns={'index':'date','5. volume':'volumn'})[['date','volumn']]
    df_api['date'] = pd.to_datetime(df_api['date']); df_api['volumn'] = pd.to_numeric(df_api['volumn'])
else:
    import yfinance as yf
    df_api = yf.download(SYMBOL, period='3mo', interval='1d').reset_index()[['Date','volumn']]
    df_api.columns = ['date','volumn']

v_api = validate(df_api, ['date','volumn']); v_api

{'missing': [], 'shape': (100, 2), 'na_total': 0}

In [13]:
_ = save_csv(df_api.sort_values('date'), prefix='api', source='alpha' if USE_ALPHA else 'yfinance', symbol=SYMBOL)

Saved data/raw/api_source-alpha_symbol-AAPL_20250820-152516.csv


## Part 2 — Scrape a Public Table (Required)
Replace `SCRAPE_URL` with a permitted page containing a simple table.

In [20]:
SCRAPE_URL = 'https://en.wikipedia.org/wiki/Dow_Jones_Industrial_Average'  
headers = {'User-Agent':'AFE-Homework/1.0'}
try:
    resp = requests.get(SCRAPE_URL, headers=headers, timeout=30); resp.raise_for_status()
    soup = BeautifulSoup(resp.text, 'html.parser')
    table = soup.find('table', {'class': 'wikitable'})
    table_rows = table.find_all('tr')

    header_cells = [th.get_text(strip=True) for th in table_rows[0].find_all('th')]
    data_rows = [
        [td.get_text(strip=True) for td in tr.find_all('td')]
        for tr in table_rows[1:]
        if tr.find_all('td')
    ]

    num_columns = len(data_rows[0]) if data_rows else 0
    df_scrape = pd.DataFrame(data_rows, columns=header_cells[:num_columns])
    df_scrape.columns = (
        pd.Series(df_scrape.columns)
        .str.replace(' ', '_', regex=False)
        .str.lower()
        .tolist()
    )
except Exception as e:
    print('Scrape failed, using inline demo table:', e)
    html = '<table><tr><th>Ticker</th><th>Price</th></tr><tr><td>AAA</td><td>101.2</td></tr></table>'
    soup = BeautifulSoup(html, 'html.parser')
    rows = [[c.get_text(strip=True) for c in tr.find_all(['th','td'])] for tr in soup.find_all('tr')]
    header, *data = [r for r in rows if r]
    df_scrape = pd.DataFrame(data, columns=header)

if 'Price' in df_scrape.columns:
    df_scrape['Price'] = pd.to_numeric(df_scrape['Price'], errors='coerce')
v_scrape = validate(df_scrape, list(df_scrape.columns)); v_scrape

{'missing': [], 'shape': (30, 6), 'na_total': 0}

In [21]:
_ = save_csv(df_scrape, prefix='scrape', site='example', table='markets')

Saved data/raw/scrape_site-example_table-markets_20250820-155302.csv


## Documentation
- API Source: (URL/endpoint/params)
- Scrape Source: (URL/table description)
- Assumptions & risks: (rate limits, selector fragility, schema changes)
- Confirm `.env` is not committed.